In [2]:
from typing import Optional, List, Dict

import numpy as np

import tritonclient.grpc as triton_grpc
import tritonclient.http as triton_http


In [38]:
%load_ext autoreload
%autoreload 2

In [8]:

# from https://github.com/lgray/hgg-coffea/blob/triton-bdts/src/hgg_coffea/tools/chained_quantile.py
class wrapped_triton:
    def __init__(
        self,
        model_url: str,
    ) -> None:
        fullprotocol, location = model_url.split("://")
        _, protocol = fullprotocol.split("+")
        address, model, version = location.split("/")

        self._protocol = protocol
        self._address = address
        self._model = model
        self._version = version

    def __call__(self, input_dict: Dict[str, np.ndarray]) -> np.ndarray:
        if self._protocol == "grpc":
            client = triton_grpc.InferenceServerClient(url=self._address, verbose=False)
            triton_protocol = triton_grpc
        elif self._protocol == "http":
            client = triton_http.InferenceServerClient(
                url=self._address,
                verbose=False,
                concurrency=12,
            )
            triton_protocol = triton_http
        else:
            raise ValueError(f"{self._protocol} does not encode a valid protocol (grpc or http)")

        # Infer
        inputs = []

        for key in input_dict:
            input = triton_protocol.InferInput(key, input_dict[key].shape, "FP32")
            input.set_data_from_numpy(input_dict[key])
            inputs.append(input)

        output = triton_protocol.InferRequestedOutput("softmax")

        request = client.infer(
            self._model,
            model_version=self._version,
            inputs=inputs,
            outputs=[output],
        )

        out = request.as_numpy("softmax")

        return out

In [71]:
batch_size = 1
# pfs = 100
# svs = 7
pfs = 128
svs = 10

# input_dict = {
#     "pf_points": np.random.rand(batch_size, 2, pfs).astype("float32"),
#     "pf_features": np.random.rand(batch_size, 19, pfs).astype("float32"),
#     "pf_mask": (np.random.rand(batch_size, 1, pfs) > 0.2).astype("float32"),
#     "sv_points": np.random.rand(batch_size, 2, svs).astype("float32"),
#     "sv_features": np.random.rand(batch_size, 11, svs).astype("float32"),
#     "sv_mask": (np.random.rand(batch_size, 1, svs) > 0.2).astype("float32"),
# }

input_dict = {
    "pf_features": np.random.rand(batch_size, 25, pfs).astype("float32"),
    "pf_vectors": np.random.rand(batch_size, 4, pfs).astype("float32"),
    "pf_mask": (np.random.rand(batch_size, 1, pfs) > 0.2).astype("float32"),
    "sv_features": np.random.rand(batch_size, 11, svs).astype("float32"),
    "sv_vectors": np.random.rand(batch_size, 4, svs).astype("float32"),
    "sv_mask": (np.random.rand(batch_size, 1, svs) > 0.2).astype("float32"),
}

# input_dict = {
#     "pf_points__0": np.random.rand(batch_size, 2, pfs).astype("float32"),
#     "pf_features__1": np.random.rand(batch_size, 19, pfs).astype("float32"),
#     "pf_mask__2": (np.random.rand(batch_size, 1, pfs) > 0.2).astype("float32"),
#     "sv_points__3": np.random.rand(batch_size, 2, svs).astype("float32"),
#     "sv_features__4": np.random.rand(batch_size, 11, svs).astype("float32"),
#     "sv_mask__5": (np.random.rand(batch_size, 1, svs) > 0.2).astype("float32"),
# }

In [74]:
# model_url = "triton+grpc://ailab01.fnal.gov:8001/particlenet_hww/1"
# model_url = "triton+grpc://prp-gpu-1.t2.ucsd.edu:8001/particlenet_hww/1"
model_url = "triton+grpc://67.58.49.52:8001/ak8_MD_vminclv2ParT_manual_fixwrap/1"
# model_url = "triton+grpc://localhost:8001/particlenet_hww_ul_4q_3q/1"
# model_url = "triton+grpc://67.58.49.52:8001/particlenet_hww_ul_4q_3q/1"
triton_model = wrapped_triton(model_url)
output = triton_model(input_dict)
print(output)

[[ 8.85230140e-04  2.15207180e-03  1.54539419e-03  4.45811916e-03
   4.36031818e-03  2.87689827e-02  4.50783918e-05  2.07938254e-04
   1.43657178e-02  1.15830921e-01  8.53165329e-05  2.84879375e-03
   6.13200059e-03  1.18893813e-02  6.13800330e-06  4.22881676e-05
   5.42556569e-02  5.57915047e-02  2.41703703e-03  2.60232016e-03
   1.62428751e-05  4.68920407e-05  4.61138058e-07  1.10413991e-01
   5.05645536e-02  1.00873776e-01  3.43784019e-02  3.10834255e-02
   1.22029074e-02  5.25399446e-02  8.64188820e-02  1.05023786e-01
   5.61055203e-04  9.55004618e-02  2.67956231e-04  8.01808725e-04
   1.06152203e-02  7.55188293e+01 -2.86533618e+00 -2.83437681e+00
   6.32742703e-01 -1.67260396e+00  8.82147968e-01  1.15630671e-01
   2.15264723e-01  2.29086235e-01  1.25586379e+00  6.61229432e-01
  -2.39497113e+00  1.61977410e-01  8.29905927e-01  1.01231778e+00
   4.86270577e-01 -3.73178899e-01  1.53381646e-01 -3.12015861e-01
  -1.14642099e-01  1.96895230e+00 -1.53730050e-01  1.35994267e+00
   3.33601

In [75]:
import onnx
import onnxruntime as ort

onnx_model = onnx.load("model.onnx")
onnx.checker.check_model(onnx_model)

ort_sess = ort.InferenceSession('model.onnx')
outputs = ort_sess.run(None, input_dict)
print(outputs)

# Print Result 
# predicted, actual = classes[outputs[0][0].argmax(0)], classes[y]
# print(f'Predicted: "{predicted}", Actual: "{actual}"')

[array([[8.85232002e-04, 2.15207506e-03, 1.54539733e-03, 4.45813034e-03,
        4.36032098e-03, 2.87691113e-02, 4.50789557e-05, 2.07940859e-04,
        1.43656582e-02, 1.15830332e-01, 8.53175952e-05, 2.84882332e-03,
        6.13200106e-03, 1.18893655e-02, 6.13805332e-06, 4.22883531e-05,
        5.42557947e-02, 5.57915345e-02, 2.41703819e-03, 2.60232110e-03,
        1.62429624e-05, 4.68914222e-05, 4.61137432e-07, 1.10414192e-01,
        5.05647399e-02, 1.00873932e-01, 3.43785509e-02, 3.10834944e-02,
        1.22029325e-02, 5.25400266e-02, 8.64187554e-02, 1.05023898e-01,
        5.61060035e-04, 9.54999030e-02, 2.67956406e-04, 8.01815011e-04,
        1.06151653e-02]], dtype=float32)]


In [68]:
outputs[0].shape

(10, 166)

In [69]:
import torch
from ParticleTransformerHidden import ParticleTransformerTagger

part_model = ParticleTransformerTagger(
    pf_input_dim=25,
    sv_input_dim=11,
    num_classes=37 + 1, # one dim for regression
    # network configurations
    pair_input_dim=4,
    embed_dims=[128, 512, 128],
    pair_embed_dims=[64, 64, 64],
    num_heads=8,
    num_layers=8,
    num_cls_layers=2,
    block_params=None,
    cls_block_params={'dropout': 0, 'attn_dropout': 0, 'activation_dropout': 0},
    fc_params=[],
    activation='gelu',
    # misc
    trim=True,
    for_inference=True,
).cpu()

In [70]:
part_model.load_state_dict(torch.load("net_best_epoch_state.pt", map_location=torch.device("cpu")))
_ = part_model.eval()

data_config = {
    "input_names": ["pf_features", "pf_vectors", "pf_mask", "sv_features", "sv_vectors", "sv_mask"],
    "input_shapes": {
        "pf_features": (-1, 25, pfs),
        "pf_vectors": (-1, 4, pfs),
        "pf_mask": (-1, 1, pfs),
        "sv_features": (-1, 11, svs),
        "sv_vectors": (-1, 4, svs),
        "sv_mask": (-1, 1, svs),
    },
}

model_info = {
    "input_names": list(data_config["input_names"]),
    "input_shapes": {k: ((1,) + s[1:]) for k, s in data_config["input_shapes"].items()},
    "output_names": ["softmax"],
    "dynamic_axes": {
        **{k: {0: "N", 2: "n_" + k.split("_")[0]} for k in data_config["input_names"]},
        **{"softmax": {0: "N"}},
    },
}

inputs = tuple(
    torch.ones(model_info["input_shapes"][k], dtype=torch.float32)
    for k in model_info["input_names"]
)
torch.onnx.export(
    part_model,
    inputs,
    "model.onnx",
    input_names=model_info["input_names"],
    output_names=model_info["output_names"],
    dynamic_axes=model_info.get("dynamic_axes", None),
    opset_version=11,
)


/opt/anaconda3/envs/python310/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/opt/anaconda3/envs/python310/lib/python3.10/site-packages/torch/nn/functional.py:5046: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert embed_dim == embed_dim_to_check, \
/opt/anaconda3/envs/python310/lib/python3.10/site-packages/torch/nn/functional.py:5053: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not gener

In [59]:
in_tensors = [torch.Tensor(val) for key, val in input_dict.items()]
in_tensors[2] = in_tensors[2].bool()
in_tensors[5] = in_tensors[5].bool()

out = part_model(*in_tensors)
print(out)

tensor([[ 1.5299e-03,  2.9319e-03,  1.9769e-03,  5.4878e-03,  7.8053e-03,
          1.2931e-02,  1.3580e-04,  6.4316e-05,  5.6223e-03,  3.5439e-03,
          5.7096e-05,  7.1246e-05,  2.4305e-03,  7.1927e-04,  4.5472e-05,
          2.3293e-05,  7.5924e-02,  3.5818e-02,  8.9561e-03,  9.3922e-03,
          4.7991e-06,  2.2116e-05,  4.7249e-07,  9.0073e-02,  4.4058e-02,
          1.9110e-01,  5.3661e-02,  8.5326e-02,  2.5597e-02,  6.6441e-02,
          1.0755e-01,  1.1822e-01,  5.6389e-04,  2.4701e-02,  1.2583e-03,
          8.0306e-04,  1.5152e-02,  1.1746e+02, -1.4494e+00, -3.8877e+00,
         -4.1724e-01, -2.1862e+00,  4.6684e-01,  2.2150e-01,  2.5049e-01,
          1.4588e+00,  5.7455e-01,  6.8124e-01, -2.3397e+00, -7.7287e-01,
          7.0747e-01,  1.9500e+00, -3.2054e-01, -3.3750e-01, -3.7859e-01,
         -3.5869e-01,  8.4656e-01,  2.2947e+00, -5.7608e-01,  1.1714e+00,
         -9.2651e-02, -8.0103e-01,  2.5926e-02,  1.4265e+00, -2.4823e+00,
          1.6136e+00,  1.8963e+00,  6.

/opt/anaconda3/envs/python310/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [53]:
output

array([[ 1.52987312e-03,  2.93191988e-03,  1.97693706e-03,
         5.48777496e-03,  7.80526781e-03,  1.29312817e-02,
         1.35801572e-04,  6.43163876e-05,  5.62225794e-03,
         3.54393735e-03,  5.70958327e-05,  7.12459951e-05,
         2.43050908e-03,  7.19271193e-04,  4.54720794e-05,
         2.32931434e-05,  7.59244114e-02,  3.58182155e-02,
         8.95608403e-03,  9.39224847e-03,  4.79914434e-06,
         2.21154678e-05,  4.72491791e-07,  9.00733918e-02,
         4.40584160e-02,  1.91101983e-01,  5.36605977e-02,
         8.53259340e-02,  2.55966950e-02,  6.64411336e-02,
         1.07548602e-01,  1.18220098e-01,  5.63886017e-04,
         2.47012507e-02,  1.25828688e-03,  8.03059200e-04,
         1.51520669e-02,  1.17462372e+02, -1.44942522e+00,
        -3.88765955e+00, -4.17238146e-01, -2.18617630e+00,
         4.66837943e-01,  2.21496135e-01,  2.50485390e-01,
         1.45877016e+00,  5.74553132e-01,  6.81239963e-01,
        -2.33971190e+00, -7.72875249e-01,  7.07470357e-0

In [51]:
out

tensor([[ 8.1155e-04,  2.0606e-03,  1.5020e-03,  2.6562e-03,  9.1622e-03,
          3.2599e-03,  1.4781e-03,  9.5434e-04,  3.3028e-04,  6.9261e-04,
          8.2008e-04,  9.9679e-04,  1.7369e-04,  2.3844e-04,  3.0242e-04,
          3.9385e-04,  4.7146e-03,  1.8696e-03,  1.0828e-03,  8.7911e-04,
          1.6338e-04,  4.5681e-05,  9.7771e-06,  1.5454e-01,  3.1454e-02,
          3.7411e-02,  1.8826e-02,  9.6202e-02,  1.1759e-02,  4.5734e-02,
          2.2030e-01,  1.1428e-01,  6.0745e-02,  5.3709e-02,  2.1316e-02,
          4.7606e-02,  5.1513e-02,  4.4246e+01, -2.1041e-02, -3.4620e+00,
         -5.6576e-02,  2.3017e-01, -4.3493e-01, -4.8431e-01,  5.6465e-02,
         -5.0499e-01,  8.3038e-02,  2.3239e-01, -1.3889e+00,  1.7120e+00,
          1.3771e-01, -1.2663e+00,  6.6230e-01,  7.4898e-02, -3.1588e-01,
          3.6870e-01, -6.8668e-01,  1.6357e+00, -8.1999e-01,  2.1713e-01,
          6.4083e-01,  8.7478e-01,  9.4118e-01, -4.9381e-01, -1.2368e+00,
          1.0730e+00,  1.7268e+00,  3.